---

<img src="../logo/anchormen-logo.svg" width="500">

---

# Exercises: solutions

## Before you start

> *This set of exercises was written to accomodate a class of very mixed ability. The difficulty level of the exercises, therefore, ranges from 'easy' to 'really quite tricky' via 'only easy once you know how'. Therefore, don't be disheartened if you run into a difficult exercise: instead, tackle it, Google it, ask colleagues for tips, study the solution, try to recreate the solution, and learn as much as possible along the way.*

In [1]:
import doctest  # Doctest
import itertools
import random
import re
import sys

from pathlib import Path


### Write a function that takes a dict as input, and returns `True` if the dict contains the key `Jill`

In [2]:
def contains_jill(mydict):
    """Does the dictionary contain an item with the key 'Pier'?
    
    The code below is a doctest: the first line with `>>> ` contains the code to test,
    and the line below contains the expected output.
    `doctest.testmod()` tests all docstrings in a module.
    
    >>> contains_jill({'Jill': 8})
    True
    
    >>> contains_jill({'Jack': 9})
    False
    
    >>> contains_jill({'Jack': 10,
    ...     'Jill': 11,
    ...     'pail of water': 12})
    True
    """
    return 'Jill' in mydict

doctest.testmod()

TestResults(failed=0, attempted=3)

Remark / extra detail: 
The reason to test with 'is True' and 'is False' is to test whether the function returns the exact object `True`/`False`. Python has a concept called 'truthiness': where Python expects `True` or `False`, such as in an `assert`- or `if`-statement, you can also pass an ordinary value that is then interpreted as True or False.

These are the main falsy values:
- `None`
- 0
- `''` (the empty string)
- `[]` (the empty list)
- `{}` (the empty dict)

Checking for truthiness vs checking for the value `True`:

    if x:           # Checks if x is truthy: True, nonempty string, nonempty list, an object, etc.
    if x is True:   # Checks if x has the value `True`
    if not x:       # Checks if x is falsy: None, False, '', [], or {} are all okay.
    if x is False:  # Checks if x has the value `False`
    
To see if Python thinks a value `x` is truthy or falsy, run `bool(x)`.

### A function that takes a dict as input, and returns a smaller dict with only those key-value pairs whose key starts with a capital

In [3]:
def keep_only_capitals(dict):
    """
    Keep only items whose key starts with a capital letter.
    
    >>> keep_only_capitals({'Pier': 'baard', 'tjoris': 'baardloos'})
    {'Pier': 'baard'}
    """
    return {key: value for key, value in dict.items() if key[0].isupper()}

doctest.testmod()

TestResults(failed=0, attempted=4)

## Exercises with `pathlib.Path`

These exercises let you practice reading the Python standard library documention. Useful for this very common situtation: "I should be able to do this with library function. Let me read the docs, find the functions I need, and figure out how to use them."

https://docs.python.org/3.6/library/pathlib.html

### A function that accepts a path as a string, and checks whether the file/directory described by that string exists

In [4]:
def file_exists(string):
    return Path(string).exists()

# `!` is a Jupyter Notebook special. It runs a command in the shell:
#     Cmd or Powershell on Windows, your usual shell (usually Bash) on Linux.
# `echo something` sends the string 'something' to standard output.
# Usually standard output is printed to the command line.
# `command > filename` redirects the command's output so it is not printed to the command line, but
#     overwrites a file.
# If the file does not exist yet, it is created: exactly what we want, because now we can test file_exists!
!echo hallo > monkey
file_exists('monkey')

True

In [5]:
# On windows: !del monkey
!rm monkey
file_exists('monkey')

False

### A function that returns a list of all files and folders in the current folder.

In [6]:
def list_contents(folder='.'):
    """
    List of all files and directories in the given folder

    Parameters
    ----------
    folder: str
        The folder whose contents you want to know. Default is '.', the current folder.

    Returns
    -------
    list of paths
        List of files and folders in the current directory
    """
    return list(Path(folder).glob('*'))

list_contents('.')

[PosixPath('.ipynb_checkpoints'),
 PosixPath('3-Python-Exercises-solutions.ipynb'),
 PosixPath('5-List-comprehension-exercises-solutions.ipynb')]

### A function that returns two lists: one with all folders in the current folder, one with all files

In [7]:
def files_and_folders(folder='.'):
    """
    Return a list of files and a list of folders in the current folder.
    
    Parameters
    ----------
    folder: str
        The folder whose contents you want to know. Default is '.', the current folder.

    Returns
    --------
    A tuple (list of files, list of folders).
    """
    contents = list_contents(folder)  # Make use of the function defined above
    files = [file for file in contents if file.is_file()]
    mappen = [file for file in contents if file.is_dir()]
    return (files, mappen)

files_and_folders()

([PosixPath('3-Python-Exercises-solutions.ipynb'),
  PosixPath('5-List-comprehension-exercises-solutions.ipynb')],
 [PosixPath('.ipynb_checkpoints')])

## Generators

### Advanced: A script that reads each line of the access log, and figures out how many bytes were sent in total.

In [8]:
!head ../data/access-log

In [9]:
def get_bytes(string):
    """Find bytes at the end of the string by matching
    a regular expression: 1 or more digits, followed by the end-of-string.
    """
    maybe_bytes = re.findall('[0-9]+$', string)  # list of 1 or empty list
    if len(maybe_bytes) == 0:
        return '0'
    else:
        return maybe_bytes[0]  # return string
    
def get_bytes_2(string):
    """
    Variant: Find bytes at the end of the string by splitting the string on whitespace and taking the last element.
    """
    # The byte count is the last element of the line, so let's just split on whitespace
    maybe_bytes = string.split()[-1]
    if maybe_bytes == '-':
        return 0
    return int(maybe_bytes)

with open('../data/access-log') as f:
    lines = f.readlines()
    str_bytes = (get_bytes(line) for line in lines)
    # If you skip this step, you'll get an error: try doing sum(['1', '2']) to see the error
    int_bytes = (int(string) for string in str_bytes)
    print(sum(int_bytes))

230741830


### Advanced: Write a function that takes one argument: a string that represents part of a filename. The function should read the access log line by line, and count how often a filename is requested that matches the pattern.

The filename is the 7th word of each log line; the part that comes after GET.

In [10]:
def count_file_occurences(pattern, file='../data/access-log'):
    """
    Parameters
    ----------
    pattern: str
        A substring of the file we are interested in
    file: str
        default '../access-log'. The file in which we should look for file occurrences.
    """
    with open(file) as f:
        lines = f.readlines()
        files = (line.split()[6] for line in lines)
        # Iterators don't support len(), because they are potentially infinite.
        # So let's map relevant files to 1, and sum the ones.
        relevant_files = (1 for file2 in files if pattern in file2)
        return(sum(relevant_files))
    
count_file_occurences('favicon')

1883

## Advanced: Convert the function above to a generator function like you saw in the slides: instead of returning a number, it should yield the lines that contain the filename pattern, one by one.

The first way to write a generator function: use `yield`, generally from inside a loop so you can yield values one by one.

In [11]:
def matching_file_generator(pattern, file='../data/access-log'):
    with open(file) as f:
        lines = f.readlines()
        # We are interested in the line, but we must inspect the line's filename.
        # So let's keep track of both
        lines_and_files = ((line, line.split()[6]) for line in lines)
        for line, filename in lines_and_files:
            if pattern in filename:
                yield line
                
# The itertools module contains all sorts of nifty tools to work with iterators
# We are going to use `itertools.islice` to get the first 6 elements,
# because slicing a generator with my_generator[:6] doesn't work.
import itertools
list(itertools.islice(matching_file_generator('jpg'), 0, 6))

# alternatively:
[next(matching_file_generator('jpg')) for i in range(6)]

['75.54.118.139 - - [24/Feb/2008:00:15:41 -0600] "GET /images/Davetubes.jpg HTTP/1.1" 200 60025\n',
 '75.54.118.139 - - [24/Feb/2008:00:15:41 -0600] "GET /images/Davetubes.jpg HTTP/1.1" 200 60025\n',
 '75.54.118.139 - - [24/Feb/2008:00:15:41 -0600] "GET /images/Davetubes.jpg HTTP/1.1" 200 60025\n',
 '75.54.118.139 - - [24/Feb/2008:00:15:41 -0600] "GET /images/Davetubes.jpg HTTP/1.1" 200 60025\n',
 '75.54.118.139 - - [24/Feb/2008:00:15:41 -0600] "GET /images/Davetubes.jpg HTTP/1.1" 200 60025\n',
 '75.54.118.139 - - [24/Feb/2008:00:15:41 -0600] "GET /images/Davetubes.jpg HTTP/1.1" 200 60025\n']

The second way to write a generator function: `yield from` an existing generator or generator expression.

In [12]:
def matching_file_generator2(pattern, file='../data/access-log'):
    with open(file) as f:
        lines = f.readlines()
        lines_and_files = ((line, line.split()[6]) for line in lines)
        yield from (line for line, filename in lines_and_files if pattern in filename)

list(itertools.islice(matching_file_generator2('jpg'), 0, 6))

['75.54.118.139 - - [24/Feb/2008:00:15:41 -0600] "GET /images/Davetubes.jpg HTTP/1.1" 200 60025\n',
 '86.132.71.214 - - [24/Feb/2008:00:37:55 -0600] "GET /images/NerdRanchEurope.jpg HTTP/1.1" 200 99542\n',
 '198.37.27.153 - - [24/Feb/2008:01:36:07 -0600] "GET /images/NerdRanchEurope.jpg HTTP/1.1" 200 99542\n',
 '201.236.226.90 - - [24/Feb/2008:09:54:24 -0600] "GET /images/Davetubes.jpg HTTP/1.1" 200 60025\n',
 '65.214.45.114 - - [24/Feb/2008:14:41:42 -0600] "GET /images/superboard.jpg HTTP/1.0" 200 71119\n',
 '75.32.37.138 - - [24/Feb/2008:16:12:30 -0600] "GET /images/Davetubes.jpg HTTP/1.1" 200 60025\n']

Bonus: how context managers work.

When you write a with-statement, a.k.a. a context manager, like this:
   
    with open(myfile) as f:
        # do stuff with f
        
    # or 
    
    with my_context_manager() as x:
        ...

This is how the context manager is implemented. Python takes care of calling `__init__`, `__enter__`, and `__exit__`. You don't need to know this right away (you can look it up when you need it), but it could help your sense that 'under the hood, it's not magic'.

In [13]:
class MyOpen:
    def __init__(self, bestandsnaam):
        """Initialisation"""
        self.bestandsnaam = bestandsnaam
        
    def __enter__(self):
        """Inside the context, we work with an open file"""
        self.open_file = open(self.bestandsnaam)
        # The return value of __enter__ is bound to x in `with MyOpen('access-log') as x:`
        return self.open_file  #
    
    def __exit__(self, exception_type, exception_value, traceback):
        """Outside the context, the file is closed again."""
        self.open_file.close()
        

with MyOpen('../data/access-log') as f:
    lines = f.readlines()
    str_bytes = (get_bytes(line) for line in lines)
    int_bytes = (int(string) for string in str_bytes)
    print(sum(int_bytes))

230741830


## Advanced: guessing game

Write a function that lets the user guess a number.

* It thinks of a number between 0 and 100.
* It uses input() to ask the user for a guess.
* If the guess is 'q', the function exits.
* It prints 'lower' or 'higher' if the true number is lower/higher than the guess.
* If the user guesses right, the function exits.

Hints: use `while True` to keep looping while the guess is wrong, `break` to break out of the loop when the guess is right, and `continue` to jump to the start of the loop.

In [14]:
import random

def guessing_game(secret_number=None):
    if secret_number is None:
        secret_number = random.randint(0, 100)

    while True:
        user_input = input('Which number from 0 to 100 am I thinking of?')
        
        # Kijk of de speler wil stoppen
        if user_input == 'q':
            print('Playtime is over')
            return  # Klaar!
        
        # Kijk of het een getal is
        try:
            guess = int(user_input)
        except ValueError:
            print("That's not a number")
            continue  # Vraag om nieuwe input
        
        # We hebben een getal; geef een hint.
        if secret_number < guess:
            print('lower...')
            continue  # Vraag om nieuwe input
        elif guess == secret_number:
            print("Supercalifragilisticexpialidocious!")
            return  # Klaar!   
        elif guess < secret_number:
            print('higher...')
            continue  # Vraag om nieuwe input
            
guessing_game()

Which number from 0 to 100 am I thinking of?3
higher...
Which number from 0 to 100 am I thinking of?80
lower...
Which number from 0 to 100 am I thinking of?50
higher...
Which number from 0 to 100 am I thinking of?60
higher...
Which number from 0 to 100 am I thinking of?67
lower...
Which number from 0 to 100 am I thinking of?65
lower...
Which number from 0 to 100 am I thinking of?64
Supercalifragilisticexpialidocious!


## Tricky to get right: number guesser

Write a function that guesses a number

* The user thinks of a number between 0 and 100
* De function prints a guess, and uses `input()` to ask the user if the guess is correct.
* The user indicates whether the number is higher or lower than the guess.
* When the user indicates the guess is correct, the function returns.

In [15]:
def getallenrader():
    at_least = 0
    at_most = 100
    while True:
        if at_most < at_least:
            print("At {lower_than} you said 'less' and at {higher_than} you said 'higher'. "
                  "That can't be true.".format(
                      lower_than=at_most + 1,
                      higher_than=at_least - 1))
            return
        guess = round((at_least + at_most) / 2)
        feedback = input('Is it {guess}? (y)es/(l)ower/(h)igher/(q)uit: '.format(guess=guess))
        if feedback.startswith('q'):
            print('farewell')
            return
        elif feedback.startswith('y'):
            print("Hooray, it's {guess}!".format(guess=guess))
            return
        elif feedback.startswith('h'):
            at_least = guess + 1
            continue
        elif feedback.startswith('l'):
            at_most = guess - 1
            continue
        else:
            continue
            
getallenrader()

Is it 50? (y)es/(l)ower/(h)igher/(q)uit: q
farewell


## Not that hard once you know how: convert the guessing_game function to a command-line script

A Python command-line script is in a `.py`-file, and has the following structure:

    import sys

    def main(command_line_arguments)
        ...

    if __name__ == '__main__':
        main(sys.argv)

Write a command-line script that can be invoked like this:

    python guessing_game.py 40

after which the user may guess the number.

Extensions for extra challenge:

* Detect when the program is invoked without a number argument; in that case, the script chooses a number on its own.

      python guessing_game.py

* Make it possible for the user to pass their name, after which the programm addresses them by name.

      python guessing_game.py 40 Draco

In [16]:
# file: guessing_game.py
import sys

HELP = """
Invoke this script as follows:
    guessing_game.py 40
where 40 is the number you want to guess at.
"""

## ---- start of guessing_game() function

## FIXME: this is where you paste the definition of guessing_game(), as seen above.

## ---- End of guessing_game() function

def validate_args(args):
    """Return the number to guess"""
    script_name, true_args = args[0], args[1:]
    if len(true_args) != 1:
        print(HELP)
        sys.exit(1)  # Exit with a non-zero exit code to signify error (not required, but many UNIX programs do it)
    try:
        return int(true_args)[0]
    except ValueError:
        print(HELP)
        sys.exit(1)  # Exit with a non-zero exit code to signify error (not required, but many UNIX programs do it)


def main(args):
    getal = validate_args(args)
    guessing_game(getal)

# __name__ is the name/import position of the current module in the program.
# When you import this file with `import guessing_game`, __name__ will be equal to "guessing_game".
# When you run this module with `python guessing_game.py`, __name__ will be equal to '__main__'.
# The code below means 'only *run* main when the user is running the script, not importing it.'
if __name__ == '__main__':
    main(sys.argv)


Invoke this script as follows:
    guessing_game.py 40
where 40 is the number you want to guess at.



SystemExit: 1

/home/angel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
